# Семинар 9. Диффузия для видео

**Преподаватель:** Никита Киселев

## Введение

На этом семинаре мы:
- Поймем, чем отличается задача генерации видео от задачи генерации изображений.
- Разберемся с тремя основными подходами к решению этой задачи.
- Адаптируем предобученную T2I диффузионную модель для имитации видео.
- Изучим возможные способы дообучения картиночной модели для получения качественных анимаций.
- Углубимся в процесс обучения T2V-диффузии с нуля, а также освоим ее расширения до задач I2V и V2V.

**План занятия:**

1. Deforum как один из способов имитации видео.
2. Адаптация предобученной T2I-модели для генерации видео с AnimateDiff.
3. CogVideoX: универсальный подход для реализации T2V, I2V и V2V генераций.


## 1. Deforum

Рассмотрим такой подход к генерации анимаций и видео, как Deforum, исходно предложенный для моделей семейства Stable Diffusion.

Прежде чем перейти к его реализации в коде и конкретным примерам, давайте зададимся вопросом о том, что вообще представляет собой полноценное видео и в чем его принципиальное отличие от набора отдельных изображений.

**Вопрос.** Что отличает видео от набора отдельных изображений?

<details>
  <summary><b>Ответ</b></summary>

  <font color='green'>Согласованность кадров во времени! Да, довольно простое и естественное требование, но как мы можем потребовать его от генеративной модели? Давайте разбираться.</font>
</details>

Допустим, вам нужно составить текстовую инструкцию, по которой далее модель формата text-to-video (T2V) сгенерирует полноценный видеоролик. Скорее всего, первое, о чем вы задумаетесь, — как именно будет происходить изменение содержимого в кадре. Без особого труда можно придумать множество таких изменений:
1. Движение объекта внутри неподвижной сцены.
2. Смена сцены вокруг неподвижного объекта.
3. Движение камеры вокруг неподвижного объекта.
4. Движение камеры и одновременное движение объекта.
5. Изменение цвета.
6. Преобразование одного объекта в другой.

Чаще всего мы представляем, что у нас есть некоторая 3D-сцена, внутри которой расположен объект в определенных внешних условиях, причем эта сцена каким-то образом изменяется. Следовательно, для генерации консистентного (согласованного) по времени набора кадров нужно **обязательно учесть физическое расположение объекта на сцене**.

Оказывается, если ограничиться только простыми физическими изменениями положения объекта в кадре, то можно использовать для генерации видео уже хорошо известную нам text-to-image (T2I) диффузионную модель, например Stable Diffusion. Один из первых подходов в этом направлении — Deforum.

### 1.1. Неподвижная сцена

Для начала рассмотрим совсем простой пример, когда сцена никак не изменяется в отношении физического расположения объекта внутри. Например, мы можем захотеть сгенерировать видеоролик, где:
1. Сначала в центре кадра сидит **кот в шляпе**.
2. Затем этого кота сменяет **собака в солнцезащитных очках**.

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1Cwqsf9Om6BPCKeRbQZsT5Le2NquFdkUw" alt="cat-dog" width="600"/>
</figure>

Поскольку любое видео — это последовательность из большого числа взаимосвязанных кадров, плавно перетекающих из одного в другой, то в качестве идеи можно попробовать:
1. Взять текстовое описание начальной сцены.
2. Взять текстовое описание финальной сцены.
3. Перевести эти текстовые описания в эмбеддинги — векторы в определенном пространстве.
4. Зафиксировать число кадров в видео, например 48, то есть 2 секунды с 24 FPS.
5. Проинтерполировать (как?) первый и последний векторы, чтобы получить 48 плавно перетекающих друг в друга векторов.
6. Для каждого из векторов сгенерировать соответствующее изображение при помощи предобученной T2I-модели.

**Вопрос.** Получится ли в таком случае согласованность между отдельными кадрами?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>Нет! Потому что мы никак не учитываем предыдущий кадр в качестве условия для генерации следующего! И как же быть? А давайте вспомним подход к редактированию изображений при помощи диффузионной модели, то есть image-to-image-преобразование.</font>
</details>

Как нам уже известно из лекции про редактирование изображений и действия с предобученными моделями, схематично подход image-to-image (I2I) пайплайна генерации можно представить так:
1. Берем исходное изображение.
2. Зашумляем его на несколько шагов, при этом **полученный шум содержит информацию об исходной картинке**.
3. Берем этот шум и начинаем действовать в обратную сторону, постепенно расшумляя картинку, при этом используем уже **новое текстовое описание**.
4. В итоге получаем изображение, похожее на исходное, но при этом согласованное с новой текстовой инструкцией!

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1D6TD3VBNqpufwlIMIFPhIH7pBweh3S2u" alt="sd-edit" width="800"/>
    <figcaption> Генерация изображения из наброска при помощи SDEdit. Синие точки показывают процесс редактирования. Зеленые и синие контуры представляют распределения картинок и набросков соответственно. Имея набросок, мы сперва зашумляем его гауссовским шумом, а затем постепенно удаляем этот шум в обратном процессе. Источник: <a href="https://arxiv.org/abs/2108.01073">Meng et al. 2021</a> </figcaption>
</figure>

Таким образом, мы пришли к тому, что представляет собой подход Deforum **для неподвижной сцены**:
1. Интерполяцию текстового описания между ключевыми кадрами.
2. Генерацию промежуточных кадров по этим проинтерполированным промптам в режиме I2I.

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1VZIvF2Cxl4uDbH030WWEMQF2syz3YU0P" alt="deforum-kandi-1" width="800"/>
    <figcaption> 1. <b>Промпт:</b> Jungle with flowers, graphic artist Magali Villeneuve, flowers, Pixar art, green and pink colors, highly detailed leaves, Lin Tinggui style, intricate oil painting, epic, delight. 2. <b>Промпт:</b> Bird of paradise painting, graphic artist Magali Villeneuve, flowers, Pixar art, green and pink colors, highly detailed leaves, Lin Tinggui style, intricate oil painting, epic, delight. <b>Режим:</b> на месте. Источник: <a href="https://habr.com/ru/companies/sberbank/articles/766968/">Kandinsky Deforum</a> </figcaption>
</figure>

### 1.2. Движение камеры вокруг объекта

Однако зачастую больший интерес для пользователей представляют динамичные анимации, в которых меняется расположение объектов, происходят повороты, сдвиги, искажения. Поэтому стоит расширить уже изученный подход на случай **движения камеры вокруг объекта**.

Как мы уже обсуждали, для этого нужно каким-то образом учесть физическое распололожение объектов на изображении.

**Вопрос.** Какой из известных вам подходов в компьютерном зрении может  справиться с такой задачей?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'> <b>Depth Estimation, или оценка глубины!</b> В рамках этого подхода создается так называемая depthmap (карта глубины) — изображение, где для каждого пикселя вместо цвета хранится его расстояние до камеры. Диапазон значений этой карты по оси $z$ (в глубину) лежит на некотором отрезке $[z_{\text{near}}, z_{\text{far}}]$ (в системе координат, о которой речь пойдет чуть ниже). Существует множество способов для получения карты глубины, в том числе и нейросетевые, например [MiDaS](https://github.com/isl-org/MiDaS) и [DepthPro](https://github.com/apple/ml-depth-pro). </font>

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1CjHsTGEKzh6hxQyGi1Kt7lKHlw9auyKV" alt="sd-edit" width="800"/>
    <figcaption> <b>Вверху:</b> исходные изображения. <b>Посередине:</b> карты глубины, построенные с помощью модели MiDaS. <b>Внизу:</b> соответствующие изображениям облака точек (отрендеренные с новых точек обзора). Источник: <a href="https://arxiv.org/abs/1907.01341">Ranftl et al. 2019</a> </figcaption>
</figure>

</details>

Так мы получаем представление о том, как именно расположены объекты в кадре (хотя и относительно неточное, поскольку с единственным изображением мы знаем сцену только с одной стороны). И далее необходимо понять, как **изменить положение камеры, чтобы соответствовать желаемому преобразованию сцены**.

Камера характеризуется координатами положения в пространстве сцены $(x, y, z)$ и направлением взгляда, которое задается тремя углами $(\alpha, \beta, \gamma)$. Соответственно, чтобы задать траекторию камеры, необходимо определить зависимости
$$
    \begin{cases}
        x = x(t), \\
        y = y(t), \\
        z = z(t), \\
    \end{cases}
    \qquad
    \begin{cases}
        \alpha = \alpha(t), \\
        \beta = \beta(t), \\
        \gamma = \gamma(t). \\
    \end{cases}
$$

Импровизированная съемка сцены камерой от первого лица представляет собой операцию перспективных проективных преобразований. Изначально камера зафиксирована в начале координат, а сцена удалена от нее на расстояние $z_{\text{near}}$.

<figure align="center">
    <img src="https://habrastorage.org/r/w1560/getpro/habr/upload_files/41f/28d/a4b/41f28da4bde433383e648e9ab393e478.png" alt="scene" width="400"/>
    <figcaption> Положение камеры и псевдо-3D-сцены в начальный момент времени. Источник: <a href="https://habr.com/ru/companies/sberbank/articles/766968/">Kandinsky Deforum</a> </figcaption>
</figure>

Важно, что движение и повороты камеры эквивалентны тем же движениям и поворотам нашей сцены, взятыми со знаком минус. Соответственно **преобразования поворота и трансляции можно применять к нашей сцене**, а камеру зафиксировать в начале координат.

Но из чего состоит сцена? Поскольку изображение — это набор пикселей, фактически и сцена для нас состоит из набора отдельных точек. Поэтому наши преобразования можно применить непосредственно к каждой из этих точек, вращая их относительно осей, проходящих через центр сцены, а затем применяя трансляцию этого центра. Вращение в трехмерном пространстве распадается на вращение вокруг трех ортогональных осей.

Представляя каждую точку нашей сцены как вектор $v_i$ с началом в центре сцены, мы получаем преобразование общего вида
$$
    f(v_i) =
    \begin{pmatrix}
        1 & 0 & 0 \\
        0 & \cos \alpha & \sin \alpha \\
        0 & -\sin \alpha & \cos \alpha \\
    \end{pmatrix}
    \begin{pmatrix}
        \cos \beta & 0 & \sin \beta \\
        0 & 1 & 0 \\
        - \sin \beta & 0 & \cos \beta \\
    \end{pmatrix}
    \begin{pmatrix}
        \cos \gamma & \sin \gamma & 0 \\
        - \sin \gamma & \cos \gamma & 0 \\
        0 & 0 & 1 \\
    \end{pmatrix}
    v_i -
    \begin{pmatrix}
        x \\
        y \\
        z
    \end{pmatrix},
$$
то есть мы последовательно
1. Поворачиваем на угол $\gamma$ вокруг оси $z$
2. Поворачиваем на угол $\beta$ вокруг оси $y$
3. Поворачиваем на угол $\alpha$ вокруг оси $x$
4. Сдвигаем на вектор с координатами $x, y, z$

Следовательно, такое преобразование применяется к каждой точке для того, чтобы посмотреть на нее с камеры $(x, y, z)$ под направлением взгляда $(\alpha, \beta, \gamma)$. Подробнее про операцию перспективного проектирования читайте в [статье](https://habr.com/ru/articles/252771/).

#### Некоторые частные случаи (примеры 2D-перспективных проекций)

<figure align="center">
    <img src="https://habrastorage.org/r/w1560/getpro/habr/upload_files/1b0/074/416/1b007441643e4be83aa3fdcfec63618a.png" alt="zoom" width="400"/>
    <figcaption> Отдаление. Источник: <a href="https://habr.com/ru/companies/sberbank/articles/766968/">Kandinsky Deforum</a> </figcaption>
</figure>

**Отдаление:** преобразование трансляции вдоль оси, проходящей через камеру и центр сцены. Трансляция производится в сторону удаления от камеры. Динамика движения камеры описывается уравнениями
$$
    \begin{cases}
        x(t) = 0, \\
        y(t) = 0, \\
        z(t) = z_0 + v \cdot t, \\
    \end{cases}
    \qquad
    \begin{cases}
        \alpha(t) = 0, \\
        \beta(t) = 0, \\
        \gamma(t) = 0, \\
    \end{cases}
$$
где $v$ — скорость движения, откуда выводится динамика каждого пикселя исходного изображения.

<figure align="center">
    <img src="https://habrastorage.org/r/w1560/getpro/habr/upload_files/bf7/597/8dc/bf75978dc4079248f9e289ef9d14e1f3.png" alt="rotation" width="400"/>
    <figcaption> Поворот. Источник: <a href="https://habr.com/ru/companies/sberbank/articles/766968/">Kandinsky Deforum</a> </figcaption>
</figure>

**Поворот:** преобразования поворота вдоль одной оси, проходящей через центр сцены. Динамика описывается следующими уравнениями:
$$
    \begin{cases}
        x(t) = 0, \\
        y(t) = 0, \\
        z(t) = z_0, \\
    \end{cases}
    \qquad
    \begin{cases}
        \alpha(t) = 0, \\
        \beta(t) = 0, \\
        \gamma(t) = \omega \cdot t, \\
    \end{cases}
$$
где $\omega$ — угловая скорость вращения, откуда выводится динамика каждого пикселя исходного изображения.

Обратите внимание, что карта глубины одной картинки все же не является полноценной 3D-сценой, и небольшие отклонения камеры от исходной проекции будут давать искажения, как это видно на изображениях выше. Поэтому есть два необходимых дополнения к этим рассуждениям:
- Каждое из последовательных преобразований должно **незначительно менять** положение сцены, то есть применяемые операции должны быть инфинитезимальными преобразованиями для получения проекции без сильного искажения.
- После каждого такого преобразования **необходимо применять подход I2I** для устранения на изображении возможных искажений.

<figure align="center">
    <img src="https://habrastorage.org/r/w1560/getpro/habr/upload_files/b8d/62f/69d/b8d62f69dd4f683a82e9f57939cafd1d.png" alt="scheme" width="600"/>
    <figcaption> Схема метода Deforum. Источник: <a href="https://habr.com/ru/companies/sberbank/articles/766968/">Kandinsky Deforum</a> </figcaption>
</figure>

В качестве иллюстрации рассмотрим, как устроены, например, операции отдаления и поворота (на очень наглядной картинке ниже).

<figure align="center">
    <img src="https://habrastorage.org/r/w1560/getpro/habr/upload_files/702/adc/bcc/702adcbcc172e109fb058b6f27914eae.png" alt="scheme" width="600"/>
    <figcaption> Отдаление и поворот с Deforum. Источник: <a href="https://habr.com/ru/companies/sberbank/articles/766968/">Kandinsky Deforum</a> </figcaption>
</figure>

А теперь давайте посмотрим на то, как выглядят видео, сгенерированные с другими движениями в Kandinsky Deforum.

<figure align="center">
    <img src="https://habrastorage.org/getpro/habr/upload_files/7ba/b08/1e8/7bab081e82312fc0068bcaab434aa847.gif" alt="deforum-kandi-2" width="1000"/>
    <img src="https://habrastorage.org/getpro/habr/upload_files/355/6c9/c50/3556c9c501f72d86cb37db5233f7a7ec.gif" alt="deforum-kandi-2" width="1000"/>
    <figcaption> Примеры генераций с различными режимами в Kandinsky Deforum. Источник: <a href="https://habr.com/ru/companies/sberbank/articles/766968/">Kandinsky Deforum</a> </figcaption>
</figure>

### 1.3. Реализация Deforum своими руками

Наконец, перейдем к тому, как устроен метод Deforum в коде. Учитывая, что каждое из допустимых преобразований сцены задается своей математической формулой, обычно реализации Deforum довольно сложные. Поэтому здесь мы сконцентрируемся на концептуальном понимании происходящего. Для этого нам подойдет и случай с неподвижной сценой.

In [ ]:
# Устанавливаем необходимые библиотеки
!pip install diffusers transformers torch

In [ ]:
# Импортируем необходимые модули
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler
from transformers import CLIPTextModel, CLIPTokenizer
import numpy as np
import cv2
from tqdm import tqdm
from PIL import Image
import imageio
from IPython.display import display, Video

In [ ]:
# Загрузка предобученной модели Stable Diffusion
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")
pipe.set_progress_bar_config(disable=True)

# Загрузка image-to-image pipeline
img2img_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
img2img_pipe.scheduler = DPMSolverMultistepScheduler.from_config(img2img_pipe.scheduler.config)
img2img_pipe = img2img_pipe.to("cuda")
img2img_pipe.set_progress_bar_config(disable=True)

# Текстовые описания начальной и финальной сцен
initial_prompt = "A beautiful sunrise over a calm lake"
final_prompt = "A serene sunset over a calm lake"

In [ ]:
# Преобразование текстовых описаний в эмбеддинги
def get_text_embeddings(prompt):
    inputs = pipe.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
    with torch.no_grad():
        text_embeddings = pipe.text_encoder(**inputs).last_hidden_state
    return text_embeddings

initial_embeddings = get_text_embeddings(initial_prompt)
final_embeddings = get_text_embeddings(final_prompt)

In [ ]:
# Число кадров в видео
num_frames = 6 * 8

# Интерполяция эмбеддингов
def interpolate_embeddings(embed1, embed2, num_frames):
    alpha = np.linspace(0, 1, num_frames)
    interpolated_embeddings = []
    for a in alpha:
        interpolated_embedding = (1 - a) * embed1 + a * embed2
        interpolated_embeddings.append(interpolated_embedding)
    return torch.stack(interpolated_embeddings)

interpolated_embeddings = interpolate_embeddings(initial_embeddings, final_embeddings, num_frames)

In [ ]:
# Генерация изображений для каждого эмбеддинга
images = []
previous_image = None

for i, embedding in enumerate(tqdm(interpolated_embeddings, desc="Generating images")):
    if i == 0:
        # Генерация первого кадра
        with torch.no_grad():
            image = pipe(prompt_embeds=embedding).images[0]
    else:
        # Генерация последующих кадров с использованием image-to-image pipeline
        with torch.no_grad():
            image = img2img_pipe(prompt_embeds=embedding, image=previous_image, strength=0.6).images[0]

    images.append(image)
    previous_image = image

In [ ]:
# Переводим PIL картинки в набор np.array
image_arrays = [np.array(img) for img in images]

# Инициализируем имя и число FPS
output_video_path = 'deforum-from-scratch.mp4'
fps = 8

# Записываем картинки в файл, используя imageio
with imageio.get_writer(output_video_path, fps=fps) as writer:
    for img_array in image_arrays:
        writer.append_data(img_array)

print(f"Video saved to {output_video_path}")

In [ ]:
display(Video(output_video_path, embed=True))

![](https://drive.google.com/uc?export=view&id=1zqeN8BkfRSz2V9LEVIEWvPcEhDLjSDm0)

Получилась интересная анимация, и для ее генерации мы не прибегали ни к какому дообучению и тем более обучению моделей. Таким образом, мы разобрали один из возможных **training-free (без обучения)** подходов к text-to-video (T2V) генерации.

## 2. AnimateDiff

Перейдем к следующему виду моделей генерации видео, который существенно отличается от предыдущего. Как мы уже отметили, предыдущий подход заключался в **использовании предобученной text-to-image (T2I) диффузионной модели**. Вполне естественно ожидать, что если **дообучить такую модель на наборе видео**, то мы получим куда лучшее качество генерации.

На самом деле сейчас все подходы к генерации видео можно разделить на три группы:
1. Использование T2I-модели без дообучения (Deforum и так далее).
2. Дообучение T2I-модели на наборе видео (AnimateDiff и так далее).
3. Обучение T2V-модели с нуля (CogVideoX, HunyuanVideo и так далее).

Постепенно мы рассмотрим каждую из возможных альтернатив, но сейчас давайте сфокусируемся на варианте с дообучением. Ниже увидите примеры видео, сгенерированных при помощи такой модели AnimateDiff.

### 2.1. Мотивация и архитектура

Авторы статьи [AnimateDiff](https://arxiv.org/abs/2307.04725) подошли к задаче генерации видео в первую очередь со стороны данных, на которых такая модель должна учиться. Совершенно понятно, что модель T2I-формата, например Stable Diffusion, училась на огромном наборе качественных пар (текст, изображение). Однако собрать датасет с хорошо размеченными видеороликами, к тому же такими, чтобы на них не было артефактов, дефектов, они были динамичными, — очень сложно.

Поэтому подход состоит из дообучения в **три последовательных этапа**:
1. Уничтожение негативных эффектов от обучения на новом наборе видео при помощи **Domain Adapter (адаптера домена)**.
2. Обучение базовым движениям с **Motion Module (модулем движения)**.
3. Адаптация к новым движениям с **MotionLoRA**.

Общая схема метода представлена ниже.

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1lWuuz0FVMgHKvLhAtX7CglFakWaZRQE2" alt="animatediff-scheme" width="1000"/>
    <figcaption> <b>Пайплайн обучения AnimateDiff</b>. AnimateDiff имеет три стадии обучения для соответствуюших модулей. 1) Сначала domain adapter обучается для того, чтобы минимизировать разницу в распределениях доменов выбранной T2I-модели и обучающего набора видео. 2) Затем вставляется motion module, который выучиывает основные базовые принципы движения из видео. 3) Наконец (опционально), MotionLoRA обучается на небольшом числе примеров, чтобы адаптироваться под движения конкретно из них. Источник: <a href="https://arxiv.org/abs/2307.04725">Guo et al. 2023</a> </figcaption>
</figure>

Далее мы пройдемся подробно по каждому блоку, но сперва давайте вспомним базовые вещи из теории диффузионных моделей, а также дообучения нейронных сетей.

**Stable Diffusion.** В качестве базовой T2I-модели авторы исходно взяли Stable Diffusion (SD). SD прозводит диффузионный процесс в латентном пространстве предобученного автоэнкодера $\mathcal{E}(\cdot)$ и $\mathcal{D}(\cdot)$. В процессе обучения закодированное изображение $\mathbf{z}_0 = \mathcal{E}(\mathbf{x}_0)$ зашумляется до $\mathbf{z}_t$ прямым процессом:
$$
    \mathbf{z}_t = \sqrt{\bar{\alpha}_t} \mathbf{z}_0 + \sqrt{1 - \bar{\alpha}_t} \boldsymbol{\epsilon}, \quad \boldsymbol{\epsilon} \sim \mathcal{N}(0, \mathbf{I}),
$$
для $t = 1, \ldots, T$, где заранее заданные $\bar{\alpha_t}$ определяют силу зашумления на шаге $t$. Нейросеть $\boldsymbol{\epsilon}_{\theta}(\cdot)$ пытается инвертировать этот процесс, предсказывая добавленный шум, при этом минимизируя функцию потерь:
$$
    \mathcal{L}(\theta) = \mathbb{E}_{\mathcal{E}(\mathbf{x}_0), \mathbf{y}, \boldsymbol{\epsilon} \sim \mathcal{N}(0, \mathbf{I}), t} \left[ \left\| \boldsymbol{\epsilon} - \boldsymbol{\epsilon}_{\theta}(\mathbf{z}_t, t, \tau_{\phi}(\mathbf{y})) \right\|_2^2 \right],
$$
где $\mathbf{y}$ — это текстовый промпт, соотвествующий $\mathbf{x}_0$, а $\tau_{\phi}(\cdot)$ — это текстовый энкодер. В Stable Diffusion нейросеть $\boldsymbol{\epsilon}_{\theta}(\cdot)$ реализована в виде U-Net. При этом на каждом из блоков down/up-сэмплинга находятся **слои само- и перекрестного внимания**, которые связывают генерацию с текстовой инструкцией. Далее они нам еще очень пригодятся!

**Вопрос.** Какие ключевые сущности есть у слоя внимания (Attention)?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>Queries, Keys, Values — проекции исходных представлений на новые пространства признаков.</font>
</details>


**Low-rank adaptation (LoRA).** Как вы можете знать, существует подход к дообучению больших моделей, который основан на низкоранговой аппроксимации линейных слоев, он называется LoRA. Вместо того чтобы учить все параметры исходной модели, LoRA для каждого линейного слоя с весами $\mathbf{W}$ добавляет две новые матрицы $\mathbf{A}$ и $\mathbf{B}$, а затем производит только их оптимизацию. Более конкретно, представьте, что вы дообучаете этот линейный слой так, что вносите изменение $\Delta \mathbf{W}$, то есть получаете новые веса
$$
    \mathbf{W}' = \mathbf{W} + \Delta \mathbf{W}.
$$
Идея в том, чтобы матрицу $\Delta \mathbf{W} \in \mathbb{R}^{m \times n}$ приблизить произведением двух матриц **маленького ранга** (отсылаем вас к теории вычислительной алгебры и разложениям матриц) $\mathbf{A} \in \mathbb{R}^{m \times r}$ и $\mathbf{B} \in \mathbb{R}^{n \times r}$:
$$
    \mathbf{W}' = \mathbf{W} + \Delta \mathbf{W} = \mathbf{W} + \mathbf{A} \mathbf{B}^\top.
$$
Причем $r$ является гиперпараметром, и вы можете самостоятельно его подбирать, опираясь на свои нужды.

**Вопрос.** Представьте, что у вас есть линейный слой $\texttt{Linear}(1028, 1028)$, к которому вы применяете дообучение с LoRA ранга $r = 16$. На сколько процентов меньше параметров нужно учить по сравнению с дообучением всего слоя?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>В таком случае матрица, которую бы вы учили в обычном случае, имела бы размер $1028 \times 1028 = 2^{10} \times 2^{10} = 2^{20} \approx 1$M параметров. Если же использовать LoRA, то число обучаемых параметров составит $1028 \times 16 + 16 \times 1028 = 2 \times 2^4 \times 2^{10} = 2^{15} \approx 30$k параметров. Таким образом, выгода от использования LoRA практически 97%! </font>
</details>

Помимо преимущества в числе обучаемых параметров (и, как следствие, меньшем объеме необходимой памяти на устройстве), есть еще несколько плюсов использования такого подхода:
- меньшая склонность к переобучению;
- меньшая вероятность сдвига домена;
- взаимозаменяемость LoRA-модулей (например, можно обучить один модуль под домен аниме, а другой — под домен картин Малевича).

**Важно отметить**, на каких именно слоях чаще всего применяется LoRA. На практике обычно используют ее на Attention-слоях: само- и перекрестного внимания. На то есть как минимум две причины: 1) эти слои линейные с точностью до нелинейного $\texttt{softmax}$ в конце; 2) именно они чаще всего содержат информацию о взаимосвязи текста и картинки, так что полезно учить именно их.

### 2.2. Domain Adapter

Поскольку мы хотим научить модель генерировать видео, мы должны сперва адаптировать ее к тому домену, из которого они взяты. В первую очередь важно отметить, что если **дообучать T2I модель на отдельных кадрах из видео**, то часто будут попадаться такие артефакты, как смазанные или низкокачественные картинки, на которых также могут быть и водяные знаки.

Авторы проводят детальное сравнение, в результате которого выясняется, что обучение дополнительного **Domain Adapter** помогает модели лучше выучить динамику видео, поскольку минимизирует зазор между доменами.

Чтобы не терять качество базовой T2I-модели, авторы предлагают использовать в качестве такового адаптера именно LoRA, которые удобно и понятно взаимозаменяемы. А именно, они вставляют LoRA в слои само- и перекрестного внимания T2I-модели Stable Diffusion.

Для примера рассмотрим проекцию query (Q). Преобразование внутренней переменной $\mathbf{z}$ после такой проекции выглядит как
$$
    Q = \mathbf{W}_Q \mathbf{z} + \texttt{AdapterLayer}(\mathbf{z}) = \mathbf{W}_Q \mathbf{z} + \alpha \cdot \mathbf{A} \mathbf{B}^\top \mathbf{z}.
$$
Здесь скаляр $\alpha = 1$ может настраиваться в процессе инференса (например, можно поставить $\alpha = 0$, чтобы обнулить эффект адаптера домена).

### 2.3. Motion Module

Чтобы смоделировать динамику вдоль временной размерности поверх предобученной T2I модели, мы должны
1. Подготовить 2D-диффузионную модель к работе с 3D-видеоданными.
2. Спроектировать такой модуль, который бы эффективно распространял информацию вдоль временной оси.

**Адаптация к 3D-данным.** Здесь все достаточно просто, поскольку по умолчанию диффузионная модель работает с изображениями **независимо**. Следовательно, базовым решением будет преобразование входной последовательности видео $\mathbf{x} \in \mathbb{R}^{b \times c \times f \times h \times w}$ в последовательность изображений $\tilde{\mathbf{x}} \in \mathbb{R}^{(b \times f) \times c \times h \times w}$.

**Дизайн самого модуля.** Последние работы в области генерации видео исследовали различные варианты дизайна архитектуры для моделирования временной компоненты. В AnimateDiff авторы адаптировали для этой цели **трансформерную архитектуру**. В нее входят несколько слоев self-attention-блоков вдоль временной оси, а также синусоидальные позиционные эмбеддинги для кодирования локации каждого кадра в анимации.

Поскольку motion module (модуль движения) никак не учитывает **пространственную взаимосвязь**, для него все $h \times w$ пикселей на самом деле равноправны. Поэтому входную последовательность кадров стоит рассматривать как такой набор матриц:
$$
    \mathbf{z}_1, \ldots, \mathbf{z}_f, \quad \mathbf{z}_i \in \mathbb{R}^{(b \times h \times w) \times c}.
$$
Таким образом, слои временного само-внимания можно представить как
$$
    \mathbf{z}_{\text{out}} = \texttt{Attention}(Q, K, V) = \texttt{softmax}(QK^\top / \sqrt{c}) \cdot V,
$$
где матрицы проекций обозначены соответственно через $Q = \mathbf{W}_Q \mathbf{z}$, $K = \mathbf{W}_K \mathbf{z}$ и $V = \mathbf{W}_V \mathbf{z}$.

### 2.4. MotionLoRA

В то время как обученный motion module понимает общие паттерны движений, возникает вопрос, когда мы хотим эффективно адаптировать нашу модель под новые движения, такие как:
- приближение камеры;
- отдаление камеры;
- поворот камеры;
- смещение камеры;
- определенные движения объекта.

Авторы предлагают **MotionLoRA** для решения этой проблемы. Архитектура на самом деле простая: они просто вешают LoRA-слои на self-attention-блоки модуля движения.

**Вопрос.** Руководствуясь примерами выше, подумайте: к каким матрицам применяются LoRA-слои?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>К матрицам $Q$, $K$ и $V$.</font>
</details>

### 2.5. Контролируемая генерация

Один из главных плюсов модели AnimateDiff в том, что такая архитектура позволяет использовать все обученные блоки **независимо и взаимозаменяемо.**

Поэтому появляется еще одна опция у AnimateDiff — генерация с дополнительным условием, например, через [ControlNet](https://arxiv.org/abs/2302.05543). Это возможно потому, что веса ControlNet также довешиваются **поверх предобученной T2I-модели**.

Таким образом, можно использовать AnimateDiff в том числе и для задач:
- генерации видео с учетом движения человека (open pose);
- стилизации видео (depth, canny).

### 2.6. Использование AnimateDiff в коде

Перейдем к инференсу — самой интересной части занятия, связанной с диффузионными моделями. Здесь все довольно просто, поскольку мы будем использовать уже известную вам библиотеку [🤗 Diffusers](https://huggingface.co/docs/diffusers/index). Вы можете ознакомиться с полным списком возможных реализаций [здесь](https://huggingface.co/docs/diffusers/api/pipelines/animatediff). Сейчас же мы сфокусируемся на text-to-video (T2V) и video-to-video (V2V) пайплайнах, которые обсудили выше.

Начнем с примера T2V-генерации с использованием базовой модели RealisticVisionV5.1.

In [ ]:
import torch
from diffusers import AnimateDiffPipeline, DDIMScheduler, MotionAdapter
from diffusers.utils import export_to_gif

# Подгружаем motion adapter
adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=torch.float16)
# Подгружаем базовую модель, основанную на SD 1.5
model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
pipe = AnimateDiffPipeline.from_pretrained(model_id, motion_adapter=adapter, torch_dtype=torch.float16)
scheduler = DDIMScheduler.from_pretrained(
    model_id,
    subfolder="scheduler",
    clip_sample=False,
    timestep_spacing="linspace",
    beta_schedule="linear",
    steps_offset=1,
)
pipe.scheduler = scheduler

# enable memory savings
pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

In [ ]:
output = pipe(
    prompt=(
        "masterpiece, bestquality, highlydetailed, ultradetailed, sunset, "
        "orange sky, warm lighting, fishing boats, ocean waves seagulls, "
        "rippling water, wharf, silhouette, serene atmosphere, dusk, evening glow, "
        "golden hour, coastal landscape, seaside scenery"
    ),
    negative_prompt="bad quality, worse quality",
    num_frames=16,
    guidance_scale=7.5,
    num_inference_steps=25,
    generator=torch.Generator("cpu").manual_seed(42),
)
frames = output.frames[0]
export_to_gif(frames, "animation.gif")

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1Zd0FXOMjBCvkhU4fSOkSyUsZj24-jYUM" alt="animation" width="400"/>
</figure>

У нас получилась классная и реалистичная генерация морских волн. Теперь давайте попробуем реализовать V2V-генерацию при помощи ControlNet-модуля.

In [ ]:
# !pip install controlnet_aux

In [ ]:
import torch
from diffusers import AnimateDiffControlNetPipeline, AutoencoderKL, ControlNetModel, MotionAdapter, LCMScheduler
from diffusers.utils import export_to_gif, load_video

# Дополнительно нам потребуется предобработать видео, прежде чем использовать его с ControlNet
from controlnet_aux.processor import ZoeDetector

# Подгружаем предобученный ControlNet
controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-depth", torch_dtype=torch.float16)

# Мы будем использовать модуль AnimateLCM для этого примера, однако можно использовать и другие
motion_adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM")

vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16)
pipe = AnimateDiffControlNetPipeline.from_pretrained(
    "SG161222/Realistic_Vision_V5.1_noVAE",
    motion_adapter=motion_adapter,
    controlnet=controlnet,
    vae=vae,
).to(device="cuda", dtype=torch.float16)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config, beta_schedule="linear")
pipe.load_lora_weights("wangfuyun/AnimateLCM", weight_name="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora")
pipe.set_adapters(["lcm-lora"], [0.8])

In [ ]:
# Для того чтобы генерировать при помощи ControlNet, нам понадобятся модели,
# которые для изображений будут возвращать condition например, карту глубины
depth_detector = ZoeDetector.from_pretrained("lllyasviel/Annotators").to("cuda")
video = load_video("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/animatediff-vid2vid-input-1.gif")
conditioning_frames = []

with pipe.progress_bar(total=len(video)) as progress_bar:
    for frame in video:
        conditioning_frames.append(depth_detector(frame))
        progress_bar.update()

In [ ]:
prompt = "a panda, playing a guitar, sitting in a pink boat, in the ocean, mountains in background, realistic, high quality"
negative_prompt = "bad quality, worst quality"

video = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_frames=len(video),
    num_inference_steps=10,
    guidance_scale=2.0,
    conditioning_frames=conditioning_frames,
    generator=torch.Generator().manual_seed(42),
).frames[0]

export_to_gif(video, "animatediff_controlnet.gif", fps=8)

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1y6KfxFzFo7qn_1gx8smWTKSY9jn47R2w" alt="animatediff_controlnet" width="400"/>
</figure>

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1HzawCorwVSM4Uykrbi7KBx2E0MxxH1Sk" alt="animatediff_controlnet" width="400"/>
</figure>

Получилось круто! В исходном видео мы видим енота, играющего на гитаре, а в сгенерированном нами — уже панду. 😀

Отметим, что AnimateDiff действительно популярная модель, поэтому для нее вы найдете великое множество различных адаптеров, которые позволят создать именно тот контент, который вы пожелаете!

## 3. CogVideoX

В завершение этого занятия мы рассмотрим **CogVideoX** — одну из ключевых моделей на  момент создания материала. Напомним, что ранее мы уже рассмотрели модели, основанные на 1) использовании предобученной T2I; 2) дообучении такой модели. CogVideoX отличается от них тем, что **обучена полностью с нуля** одновременно **и на картинках, и на видео**.

Поскольку здесь мы ставим задачу обучить модель с нуля, важно учесть каждую из ее частей по отдельности, а именно:
- автоэнкодер, который преобразовывает видео в латентные переменные и обратно;
- модель предсказания шума в латентном пространстве.

Далее мы рассмотрим каждую из составляющих более детально, но сейчас для начала вдохновимся красивыми генерациями CogVideoX.

> An elderly gentleman, with a serene expression, sits at the water's edge, a steaming cup of tea by his side. He is engrossed in his artwork, brush in hand, as he renders an oil painting on a canvas that's propped up against a small, weathered table. The sea breeze whispers through his silver hair, gently billowing his loose-fitting white shirt, while the salty air adds an intangible element to his masterpiece in progress. The scene is one of tranquility and inspiration, with the artist's canvas capturing the vibrant hues of the setting sun reflecting off the tranquil sea.

![](https://drive.google.com/uc?export=view&id=1Zlwd1oI5n6hWPLSZoEyqfMza0E1zZi0w)

> A garden comes to life as a kaleidoscope of butterflies flutters amidst the blossoms, their delicate wings casting shadows on the petals below. In the background, a grand fountain cascades water with a gentle splendor, its rhythmic sound providing a soothing backdrop. Beneath the cool shade of a mature tree, a solitary wooden chair invites solitude and reflection, its smooth surface worn by the touch of countless visitors seeking a moment of tranquility in nature's embrace.

![](https://drive.google.com/uc?export=view&id=1YFzaQZqqNC03SNSIW7tGSAHmWf_XtbAu)

> A golden retriever, sporting sleek black sunglasses, with its lengthy fur flowing in the breeze, sprints playfully across a rooftop terrace, recently refreshed by a light rain. The scene unfolds from a distance, the dog's energetic bounds growing larger as it approaches the camera, its tail wagging with unrestrained joy, while droplets of water glisten on the concrete behind it. The overcast sky provides a dramatic backdrop, emphasizing the vibrant golden coat of the canine as it dashes towards the viewer.

![](https://drive.google.com/uc?export=view&id=1PT2I-Ji3KafMXwAN9KFCGYCCWanBjZt7)

> A suited astronaut, with the red dust of Mars clinging to their boots, reaches out to shake hands with an alien being, their skin a shimmering blue, under the pink-tinged sky of the fourth planet. In the background, a sleek silver rocket, a beacon of human ingenuity, stands tall, its engines powered down, as the two representatives of different worlds exchange a historic greeting amidst the desolate beauty of the Martian landscape.

![](https://drive.google.com/uc?export=view&id=1APVP-f61f2n65QpekbKZFD22R1YnBKgZ)

> In a dimly lit bar, purplish light bathes the face of a mature man, his eyes blinking thoughtfully as he ponders in close-up, the background artfully blurred to focus on his introspective expression, the ambiance of the bar a mere suggestion of shadows and soft lighting.

![](https://drive.google.com/uc?export=view&id=1PGLP-iRBSB4WQoFRkWABKq7cyImtFPMw)

> On a brilliant sunny day, the lakeshore is lined with an array of willow trees, their slender branches swaying gently in the soft breeze. The tranquil surface of the lake reflects the clear blue sky, while several elegant swans glide gracefully through the still water, leaving behind delicate ripples that disturb the mirror-like quality of the lake. The scene is one of serene beauty, with the willows' greenery providing a picturesque frame for the peaceful avian visitors.

![](https://drive.google.com/uc?export=view&id=1XcXmDzlmeUqAdk73COIFNnkPe17Qw8Sa)

Итак, давайте же разберемся, что именно позволяет CogVideoX генерировать настолько качественные и детализированные видео только лишь по текстовым инструкциям.

### 3.1. 3D Causal Vae

Мы начнем с разбора архитектуры автоэнкодера как одной из главных составляющих качественной диффузионной модели. Так как видео содержат и пространственную, и временную информацию, то **использование автоэнкодера совершенно необходимо**, иначе объем данных был бы просто огромным.

Чтобы справиться с вычислительными сложностями моделирования видео данных, авторы [CogVideoX](https://arxiv.org/abs/2408.06072) предлагают использовать 3D вариационный автоэнкодер (3D VAE). Идея состоит в том, чтобы внедрить для сжатия видео **3D сверточные слои**, чтобы сжимать по обеим размерностям — пространственной и временной. Это позволяет достичь большего сжатия с максимальным качеством и непрерывностью в реконструкции видео.

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1sCkMZpEWl5KeBshqCfwjCHyEJ2QjYjrZ" alt="cogvideox-vae" width="1000"/>
    <figcaption> (a) Структура 3D VAE в CogVideoX. Он состоит из энкодера, декодера и латентного регуляризатора, при этом обеспечивая $8 \times 8 \times 4$ сжатие из пиксельного пространства в латентное. (b) Реализация временной причинной свертки с контекстной параллелизацией.  Источник: <a href="https://arxiv.org/abs/2408.06072">Yang et al. 2024</a> </figcaption>
</figure>

Схема выше иллюстрирует структуру предлагаемого 3D VAE:
- Он состоит из энкодера, декодера и регуляризатора латентного пространства — Kullback—Leibler (KL) regularizer.
- Энкодер и декодер состоят из симметричных блоков, соответственно производящих $2\times$ downsampling и upsampling, между которыми располагаются промежуточные ResNet-блоки.
- Некоторые блоки также осуществляют 3D downsampling (upsampling), в то время как остальные — только 2D.

Авторы применяют **сверточные слои с временной причинной связью**, которые добавляют паддинги в начало сверточного пространства. Благодаря этому **информация из будущего не влияет на предсказание настоящего или тем более прошлого**.

Также используется уже и чисто технический трюк, который позволяет уменьшить время коммуникации между отдельными GPU при обучении. Идея в применении **контекстной параллелизации по временной размерности для 3D сверток**, чтобы распределить вычисления между девайсами. Из-за причинной природы таких сверток каждая GPU просто отправляет сегмент длины $k-1$ следующей GPU, где $k$ обозначает размер ядра по времени.

### 3.2. Expert Transformer

Интересно, что большинство state-of-the-art моделей генерации видео используют именно трансформерную архитектуру для предсказания шума, в том числе и CogVideoX. Давайте разберемся, как именно происходит обработка входных данных в таком трансформере.

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=197pCgPWHnfiTQy4F4EJ3Rncdb4VawgVE" alt="cogvideox-vae" width="500"/>
    <figcaption> Полная архитектура CogVideoX. Источник: <a href="https://arxiv.org/abs/2408.06072">Yang et al. 2024</a> </figcaption>
</figure>

**Patchify.** Как вы знаете из занятия про современные архитектуры в диффузии, любой диффузионный трансформер (DiT) начинает свою работу с разбиения входного латента на отдельные части — патчи. Так, 3D VAE кодирует видео в латент размера $T \times H \times W \times C$, а затем слой патчификации разбивает его на последовательность $\mathbf{z}_{\text{vision}}$ длины $\frac{T}{q} \cdot \frac{H}{p} \cdot \frac{W}{p}$.

**3D-RoPE.** В трансформерных моделях позиционные эмбеддинги используются для добавления информации о порядке токенов в последовательности. Это необходимо, так как самовнимание (self-attention) в Transformer не учитывает порядок токенов. Обычные позиционные эмбеддинги добавляются к входным эмбеддингам токенов, чтобы модель могла учитывать порядок токенов в последовательности. Пример обычного позиционного эмбеддинга:
$$
    \text{Input Embedding} + \text{Positional Embedding}
$$

[Rotary Positioned Embeddings](https://arxiv.org/abs/2104.09864) (RPE) — это более сложный метод добавления позиционной информации к эмбеддингам токенов. Вместо того чтобы просто добавлять позиционные эмбеддинги к входным эмбеддингам, RPE использует вращение (rotation) в пространстве эмбеддингов.

Как это работает:
- **Разделение эмбеддингов.** Эмбеддинги токенов разделяются на две части, которые можно интерпретировать как координаты в комплексном пространстве.
- **Вращение.** Каждая позиция в последовательности ассоциируется с углом вращения. Эмбеддинги токенов вращаются на этот угол в комплексном пространстве.
- **Комбинирование.** Вращенные эмбеддинги используются в модели вместо исходных эмбеддингов.

Формально, если у нас есть эмбеддинг токена $\mathbf{x}$ и позиционный эмбеддинг $\mathbf{p}$, то RPE можно описать следующим образом:
$$
    \mathbf{x}' = R(\theta) \cdot \mathbf{x},
$$
где $R(\theta)$ — матрица вращения, зависящая от позиции $\theta$.

Основное преимущество RPE — более гладкая интерполяция, так как они позволяют более плавно интерполировать позиционную информацию.


**Expert Adaptive Layernorm.** Авторы конкатенируют текстовые и видеоэмбеддинги, чтобы лучше выравнить их семантическую близость. Однако, поскольку пространства признаков текста и видео все же сильно отличаются, их эмбеддинги, скорее всего, имеют различный масштаб. Чтобы учесть это, предлагается использовать, как и в оригинальном [DiT](https://arxiv.org/abs/2212.09748), модулирование сигнала путем подачи временного шага $t$ из диффузионного процесса на вход механизма модуляции.

Затем **Vision** Expert Adaptive Layernorm и **Text** Expert Adaptive Layernorm применяют эти масштабирования к промежуточным состояниям и таким образом уравновешивают признаки.

**3D Full Attention.** Несмотря на то, что большинство предыдущих работ обычно разделяли внимание на пространственную и временную размерности, авторы CogVideoX решили от этого отказаться в пользу лучшего качества модели, используя полноценный 3D Full Attention.

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1O9KeQuRSelnrOu6Aq5EnbadR7LuhDoIm" alt="cogvideox-vae" width="500"/>
    <figcaption> Раздельные слои пространственного и временного внимания не справляются с полноценным учетом движения в кадре. Так, на рисунке голова человека на кадре i+1 не может явно "обратить внимание" на нее же, но на i-м кадре. Источник: <a href="https://arxiv.org/abs/2408.06072">Yang et al. 2024</a> </figcaption>
</figure>

Конечно, такой подход более мощный, поскольку позволяет учесть многие краевые эффекты (см. картинку выше), однако он требует значительно больших вычислений, а потому и время работы модели сильно увеличивается.

### 3.3. Использование CogVideoX в коде

Наконец, мы можем перейти от теории к реализации. Здесь мы, как обычно, воспользуемся нашей любимой библиотекой [🤗 Diffusers](https://huggingface.co/docs/diffusers/index) и импортируем уже заранее подготовленные пайплайны:
- Text-to-Video
- Image-to-Video
- Video-to-Video

В процессе проговорим каждый из них чуть подробнее.

#### 3.3.1. Text-to-Video

Здесь все довольно просто — пайплайн такой же, как мы обсуждали ранее. Подаем текстовое описание, а модель на выходе генерирует соответствующее ему видео.

In [ ]:
!pip install diffusers transformers hf_transfer

In [ ]:
# !pip install git+https://github.com/huggingface/accelerate
!pip install accelerate==0.33.0

Импортируем необходимые библиотеки. Кусок кода ниже опционален, однако если его включить, то подгрузка модели с HF Hub будет быстрее.

In [ ]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [ ]:
import torch
from diffusers import (
    AutoencoderKLCogVideoX,
    CogVideoXTransformer3DModel,
    CogVideoXPipeline,
    CogVideoXImageToVideoPipeline,
    CogVideoXVideoToVideoPipeline,
    CogVideoXDPMScheduler
)
from diffusers.utils import export_to_video, load_image, load_video
from transformers import T5EncoderModel
from IPython.display import display, Video

Подгружаем модели и создаем пайплайн.

**Примечание.** `bfloat16`, который является рекомендованным `dtype` для CogVideoX-5B, вызовет ошибку Out Of Memory в виду недостаточной поддержки на Turing GPUs. Поэтому мы будем использовать `float16`, который, к сожалению, может привести к более плохому качеству генерации. Лучше всего использовать Ampere или GPUs , но имеем то, что имеем...

In [ ]:
# Спасибо [@camenduru](https://github.com/camenduru)!
# Причина, по которой мы используем чекпоинт Camenduru вместо оригинального, состоит в том, что они
# были экспортированы с max_shard_size в размере "5GB", когда сохранялись с `.save_pretrained`.
# Оригинальные же веса были с max_shard_size в размере "10GB", что на Colab привело бы к OOM (на CPU).

transformer = CogVideoXTransformer3DModel.from_pretrained("camenduru/cogvideox-5b-float16", subfolder="transformer", torch_dtype=torch.float16)
text_encoder = T5EncoderModel.from_pretrained("camenduru/cogvideox-5b-float16", subfolder="text_encoder", torch_dtype=torch.float16)
vae = AutoencoderKLCogVideoX.from_pretrained("THUDM/CogVideoX-5b", subfolder="vae", torch_dtype=torch.float16)

In [ ]:
# Создаем пайплайн и запускаем инференс
pipe = CogVideoXPipeline.from_pretrained(
    "THUDM/CogVideoX-5b",
    text_encoder=text_encoder,
    transformer=transformer,
    vae=vae,
    torch_dtype=torch.float16,
)

Подключаем оптимизации памяти. Отметим, что очень важно использование последовательной GPU offloading, чтобы запускать модель на Turing и ниже архитектурах GPU. Это агрессивно переносит практически все на CPU и перемещает на GPU только то, что нужно именно в моменте. Это сохраняет очень много VRAM, однако приводит к сильно долгим генерациям... Но что поделать на Colab...

Еще обратим ваше внимание на такую технику, как tiling, или разбиение на фрагменты во время энкодинга и декодинга VAE. Идея в том, что в отличие от обработки изображений (2D данных), преобразование фрагмента видео (3D данных) требует куда большее число памяти за раз, из-за чего не всегда удается даже просто сделать forward pass у энкодера или декодера VAE. Tiling — это процедура, при которой объект разбивается по пространственной размерности на отдельные фрагменты (тайлы), перекрывающие друг друга с небольшим запасом, которые обрабатываются последовательно, а затем конкатенируются для получения итогового результата. При этом накладывающиеся друг на друга края смешиваются (происходит blending).

In [ ]:
pipe.enable_sequential_cpu_offload()
# pipe.vae.enable_tiling()

Генерируем!

In [ ]:
prompt = (
    "A panda, dressed in a small, red jacket and a tiny hat, sits on a wooden stool in a serene bamboo forest. "
    "The panda's fluffy paws strum a miniature acoustic guitar, producing soft, melodic tunes. Nearby, a few other "
    "pandas gather, watching curiously and some clapping in rhythm. Sunlight filters through the tall bamboo, "
    "casting a gentle glow on the scene. The panda's face is expressive, showing concentration and joy as it plays. "
    "The background includes a small, flowing stream and vibrant green foliage, enhancing the peaceful and magical "
    "atmosphere of this unique musical performance."
)

In [ ]:
video = pipe(prompt=prompt, guidance_scale=6, use_dynamic_cfg=True, num_inference_steps=50).frames[0]

In [ ]:
export_to_video(video, "output-t2v.mp4", fps=8)

In [ ]:
display(Video("output-t2v.mp4", embed=True))

![](https://drive.google.com/uc?export=view&id=1sHYyrwws_UxHl6BvHsrvlxN15nMbYbyW)

Итак, у нас получилось сгенерировать видео с помощью CogVideoX в формате Text-to-Video.

#### 3.3.2. Image-to-Video

Этот пайплайн уже посложнее, поскольку просто так, без дообучения, работать не будет. Авторы модифицируют архитектуру, **конкатенируя исходный латент с латентом картинки**. Благодаря этому модель учится использовать картинку в качестве первого кадра видео.

In [ ]:
# Инициализируем части пайплайна
transformer = CogVideoXTransformer3DModel.from_pretrained("THUDM/CogVideoX-5b-I2V", subfolder="transformer", torch_dtype=torch.float16)
text_encoder = T5EncoderModel.from_pretrained("THUDM/CogVideoX-5b-I2V", subfolder="text_encoder", torch_dtype=torch.float16)
vae = AutoencoderKLCogVideoX.from_pretrained("THUDM/CogVideoX-5b-I2V", subfolder="vae", torch_dtype=torch.float16)

In [ ]:
# Создаем пайплайн и запускаем инференс
pipe = CogVideoXImageToVideoPipeline.from_pretrained(
    "THUDM/CogVideoX-5b-I2V",
    text_encoder=text_encoder,
    transformer=transformer,
    vae=vae,
    torch_dtype=torch.float16,
)

In [ ]:
pipe.enable_sequential_cpu_offload()
# pipe.vae.enable_tiling()

In [ ]:
prompt = "An astronaut hatching from an egg, on the surface of the moon, the darkness and depth of space realised in the background. High quality, ultrarealistic detail and breath-taking movie-like camera shot."
image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/astronaut.jpg")
images.show()

![](https://drive.google.com/uc?export=view&id=1r7ANGgIaYIrBSKbF7cAUcDSc22aTDXAh)

In [ ]:
video = pipe(image=image, prompt=prompt, guidance_scale=6, use_dynamic_cfg=True, num_inference_steps=50).frames[0]

In [ ]:
export_to_video(video, "output-i2v.mp4", fps=8)

In [ ]:
display(Video("output-i2v.mp4", embed=True))

![](https://drive.google.com/uc?export=view&id=1A70FqReIK5c0ANwV6HVCtEbsACpLSBgb)

Итак, у нас получилось сгенерировать видео с помощью CogVideoX в формате Image-to-Video.

#### 3.3.3. Video-to-Video

В отличие от Image-to-Video, пайплайн Video-to-Video работает без дообучения. Идея такая же, как и в случае image-to-image генерации изображений (см. примеры из Deforum выше). Мы частично зашумляем входное видео, а затем стартуем с этой промежуточной точки на траектории и расшумляем уже с новым текстовым описанием.

In [ ]:
# Создаем пайплайн и запускаем инференс
pipe = CogVideoXVideoToVideoPipeline.from_pretrained(
    "THUDM/CogVideoX-5b",
    text_encoder=text_encoder,
    transformer=transformer,
    vae=vae,
    torch_dtype=torch.float16,
)
pipe.scheduler = CogVideoXDPMScheduler.from_config(pipe.scheduler.config)

In [ ]:
pipe.enable_sequential_cpu_offload()
pipe.vae.enable_tiling()

In [ ]:
input_video = load_video(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/hiker.mp4"
)
prompt = (
    "An astronaut stands triumphantly at the peak of a towering mountain. Panorama of rugged peaks and "
    "valleys. Very futuristic vibe and animated aesthetic. Highlights of purple and golden colors in "
    "the scene. The sky is looks like an animated/cartoonish dream of galaxies, nebulae, stars, planets, "
    "moons, but the remainder of the scene is mostly realistic."
)

In [ ]:
video = pipe(video=input_video, prompt=prompt, strength=0.7, guidance_scale=6, use_dynamic_cfg=True, num_inference_steps=50).frames[0]

In [ ]:
export_to_video(input_video, "input.mp4", fps=8)
export_to_video(video, "output-v2v.mp4", fps=8)

In [ ]:
display(Video("input.mp4", embed=True))
display(Video("output-v2v.mp4", embed=True))

![](https://drive.google.com/uc?export=view&id=16f3sk0p5GNsSVQH-x9tJXmBZIlxXkv1c)
![](https://drive.google.com/uc?export=view&id=1-E-2wOeGx6euHAY2DXYz1stIAQDaNm9B)


Итак, у нас получилось сгенерировать видео с помощью CogVideoX в формате Video-to-Video.

## Итоги

Подведем итоги нашего сегодняшнего занятия.
1. Есть три основных подхода к генерации видео: 1) с использованием предобученной text-to-image (T2I) модели; 2) дообучение T2I; 3) обучение T2V-модели с нуля.
2. Одним из первых подходов к генерации видео без обучения является Deforum. Идея состоит в последовательном использовании image-to-image (I2I) редактирования изображений. При этом также используются различные пространственные преобразования для изменения положения камеры.
3. Дообучение T2I-модели приводит к лучшему пониманию динамики сцены, а также более высокому качеству генерации. Один из ключевых подходов AnimateDiff позволяет достичь этого благодаря трем модулям: 1) Domain Adapter; 2) Motion Module; 3) MotionLoRA.
4. Одна из лучших моделей генерации видео на текущий момент, CogVideoX, имеет трансформерную архитектуру и обрабатывает латенты с помощью 3D Causal Vae. Большое число качественных данных и многостадийное обучение позволили CogVideoX получить отличные генерации во всех трех режимах: T2V, I2V и V2V.

#### Полезные источники

Deforum:
- https://github.com/deforum-art/sd-webui-deforum
- https://reticulated.net/dailyai/stable-diffusion-videos-with-visions-of-chaos-deforum-animation/
- https://dreamingcomputers.com/deforum-stable-diffusion/deforum-stable-diffusion-settings/
- https://medium.com/@louis24/deforum-tutorial-94b15dea53d1
- https://docs.google.com/document/d/1RrQv7FntzOuLg4ohjRZPVL7iptIyBhwwbcEYEW2OfcI/edit#heading=h.7z6glzthkva2
- https://github.com/XLabs-AI/deforum-x-flux
- https://habr.com/ru/companies/sberbank/articles/766968/
- https://colab.research.google.com/github/deforum-art/deforum-stable-diffusion/blob/main/Deforum_Stable_Diffusion.ipynb#scrollTo=IJjzzkKlWM_s
- https://huggingface.co/docs/transformers/tasks/monocular_depth_estimation

AnimateDiff:
- https://animatediff.github.io/
- https://huggingface.co/docs/diffusers/main/api/pipelines/animatediff
- https://colab.research.google.com/github/dmarx/notebooks/blob/main/AnimateDiff.ipynb
- https://github.com/huggingface/diffusers/tree/main/examples/community#animatediff-controlnet-pipeline
- https://stable-diffusion-art.com/animatediff/

CogVideoX:
- https://github.com/THUDM/CogVideo
- https://huggingface.co/spaces/THUDM/CogVideoX-5B-Space
- https://huggingface.co/docs/diffusers/api/pipelines/cogvideox
